# 数据操作的练习

本文通过实例来介绍不同数据操作方法，同时比较性能差异。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 数据集

用电需求数据集，包括两个字段：
- `date_time`，日期类型，包括2013年全年
- `energy_kwh`，每个时段的用电量

首先导入数据：

In [2]:
import os

demand_df = pd.read_csv(os.path.join('..', 'data', 'demand_profile.csv'))

查看数据集概况；

In [3]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
date_time     8760 non-null object
energy_kwh    8760 non-null float64
dtypes: float64(1), object(1)
memory usage: 137.0+ KB


In [4]:
demand_df.describe()

,energy_kwh
count,8760.000000
mean,0.653600
std,0.453193
min,0.000000
25%,0.285000
50%,0.609000
75%,0.941000
max,3.832000


## 转换日期数据类型

数据集`date_time`列为字符串类型，可以使用`pd.to_datetine()`来进行转换。显示指定格式，其转换性能会大幅提升：

In [5]:
# 深拷贝
df = demand_df.copy()

In [6]:
%%time
df['date_time'] = pd.to_datetime(df['date_time'])

Wall time: 1.21 s


In [7]:
%%time
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')

Wall time: 2.01 ms


## 计算电费

在数据集中，假定不同时间段电费费率不同，例如：
- `0--7`时间段，费率为12
- `7--17`时间段，费率为20
- `17--24`时间段，费率为28

定义如下函数来计算不同时间对应的电费：

In [8]:
def apply_tariff(kwh, hour):
    """Calculates cost of electricity for given hour."""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

下面介绍使用不同方法来计算电费。

### 使用遍历循环

使用对象的`.iterrows()`方法来进行遍历循环：

In [9]:
def apply_tariff_iterrows(df):
    energy_cost_list = []
    for index, row in df.iterrows():
        # Get electricity used and hour of day
        energy_used = row['energy_kwh']
        hour = row['date_time'].hour
        # Append cost list
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    
    df['cost_cents'] = energy_cost_list

In [10]:
%%timeit 
apply_tariff_iterrows(df)

857 ms ± 54.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 使用矢量化函数 `df.apply()`

使用`df.apply()`来进行实例化操作：

In [11]:
def apply_tariff_withapply(df):
    df['cost_cents'] = df.apply(
        lambda row: apply_tariff(
            kwh=row['energy_kwh'],
            hour=row['date_time'].hour),
        axis=1)

In [12]:
%%timeit
apply_tariff_withapply(df)

263 ms ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 矢量化操作

利用 `df.isin()` 方法来进行矢量化运算：

In [13]:
def apply_tariff_isin(df):
    # Define hour range Boolean arrays
    peak_hours = df.date_time.dt.hour.isin(range(17, 24))
    shoulder_hours = df.date_time.dt.hour.isin(range(7, 17))
    off_peak_hours = df.date_time.dt.hour.isin(range(0, 7))

    # Apply tariffs to hour ranges
    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12

In [14]:
%%timeit
apply_tariff_isin(df)

16 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 使用分组函数 `pd.cut()`

使用 Pandas 的数据分组函数`pd.cut()`（在下一章节会有详细介绍）进行运算操作：

In [15]:
def apply_tariff_cut(df):
    bins = [0, 7, 17, 24]
    cents_per_kwh = pd.cut(df.date_time.dt.hour, bins,  include_lowest=True, labels=[12, 20, 28])
    
    df['cost_cents'] = df['energy_kwh'] * cents_per_kwh.astype(int)

In [16]:
%%timeit
apply_tariff_cut(df)

3.64 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 使用 Numpy

下面使用 Numpy 函数`np.digitize()`来计算电费：

In [17]:
def apply_tariff_digitize(df):
    prices = np.array([12, 20, 28])
    bins = np.digitize(df.date_time.dt.hour.values, bins=[7, 17, 24])
    df['cost_cents'] = prices[bins] * df['energy_kwh'].values

In [18]:
%%timeit
apply_tariff_digitize(df)

1.19 ms ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 小结

Pandas 的数据运算操作有多种方法，尽量使用矢量化操作方法能大大提高性能。